In [1]:
import numpy as np
import pandas as pd
from glob import glob
import os
import scipy.io as sio
import mat73
import matplotlib.pyplot as plt

In [64]:
def load_waveform(fname):
    data = sio.loadmat(fname)
    k = list(data.keys())[-1]
    data = data[k]
    return pd.DataFrame(dict(
        fname=[x[0] for x in data['fname'][0]],
        stimulus_marker=[x[0][0] for x in data['marker'][0]],
        sampling_rate=[x[0][0] for x in data['samprate'][0]],
        resistance=[x[0][0] for x in data['RO'][0]],
        capacitance=[x[0][0] for x in data['CO'][0]],
        amplitude_modulation=[x[0][0] for x in data['amp_mod'][0]],
        waveform_modulation=[x[0][0] for x in data['wav_mod'][0]],
        value_max=[x[0][0] for x in data['maxv'][0]],
        value_min=[x[0][0] for x in data['minv'][0]],
        waveform=[x.flatten() for x in data['waveform'][0]],
        base_waveform=[data["waveform"][0][-1].flatten()] * len(data["waveform"][0]),
    ))
    

def load_lfp_data(fname):
    if fname[-4:] == ".mat":
        fname = "-".join(fname.split("-")[:-1])
    data_means = mat73.loadmat(fname + "-lfp_means.mat")["LfpMeans"]
    data_stats = mat73.loadmat(fname + "-lfp_stats.mat")["LfpStats"]

    lfp_means_time = data_means["lfptime"][-1]
    lfp_sampling_rate = data_means["vdt"][-1]
    data_means = pd.DataFrame(
        dict(
            stimulus_marker=[int(x) for x in data_means["marker"][:-1]],
            number_bouts=[int(x) for x in data_means["bouts"][:-1]],
            lfp_trace=data_means["lfpNorm"][:-1],
            mean_lfp_trace=data_means["lfpMean"][:-1],
            base_lfp_trace=data_means["b1lfpNorm"][:-1],
            base_mean_lfp_trace=data_means["b1lfpMean"][:-1],
            stimulus_amplitude_modulation=data_means["ampmod"][:-1],
            stimulus_waveform_modulation=data_means["wavmod"][:-1],
            lfp_sampling_rate=[lfp_sampling_rate] * len(data_means["marker"][:-1]),
            lfp_times = [lfp_means_time] * len(data_means["marker"][:-1]),
        )
    )
    data_stats = pd.DataFrame(
        dict(
            stimulus_marker=[int(x) for x in data_stats["marker"][:-1]],
            number_bouts=[int(x) for x in data_stats["bouts"][:-1]],
            lfp_response=data_stats["n1"][:-1],
            base_lfp_response=data_stats["b1n1"][:-1],
            mean_response_modulation=data_stats["aveNn1m"][:-1],
        )
    )
    return pd.merge(data_means, data_stats, on=["stimulus_marker", "number_bouts"])

In [65]:
data = pd.DataFrame()
for folder in glob('raw/*', recursive=True):
    if os.path.isdir(folder):
        print(folder)
        fnames = glob(f"{folder}/*lfp_means.mat")
        waveforms_fname = glob(f"{folder}/waveform*.mat")
        if len(waveforms_fname) == 1:
            waveforms_fname = waveforms_fname[0]
            waveforms = load_waveform(waveforms_fname)
            print(waveforms["waveform"][0].shape)
            new_lfp_data = pd.DataFrame()
            for fname in fnames:
                print(fname)
                experiment_date, session_id, zone, _ = fname.split('/')[-1].split('-')
                lfp_data = load_lfp_data(fname)
                new_lfp_data = pd.concat([new_lfp_data, lfp_data], axis=0, ignore_index=True)
            new_lfp_data["experiment_date"] = experiment_date
            new_lfp_data["session_id"] = session_id
            new_lfp_data["zone"] = zone
            new_lfp_data["paired_experiment"] = "paired" in folder
            new_lfp_data = pd.merge(new_lfp_data, waveforms, on="stimulus_marker")
            print()
        else:
            print(f"Folder {folder} does not contain a waveform file or contains more than one waveform file.\n")
            continue
        data = pd.concat([data, new_lfp_data], axis=0, ignore_index=True)

raw/fish_03-20190711-separate
(4201,)
raw/fish_03-20190711-separate/20190711-001-dlz-lfp_means.mat


ERROR:root:ERROR: MATLAB type not supported: LinearModel, (uint32)


raw/fish_03-20190711-separate/20190711-004-mz-lfp_means.mat


ERROR:root:ERROR: MATLAB type not supported: LinearModel, (uint32)



raw/fish_12-20200826-paired
(3564,)
raw/fish_12-20200826-paired/20200826-004-mz-lfp_means.mat


...

ERROR:root:ERROR: MATLAB type not supported: LinearModel, (uint32)


raw/fish_11-20200806-paired/20200806-001-dlz-lfp_means.mat


ERROR:root:ERROR: MATLAB type not supported: LinearModel, (uint32)



raw/fish_05-20190910-mz
(3564,)
raw/fish_05-20190910-mz/20190910-010-mz-lfp_means.mat


ERROR:root:ERROR: MATLAB type not supported: LinearModel, (uint32)


In [66]:
data.head()

,stimulus_marker,number_bouts,lfp_trace,mean_lfp_trace,base_lfp_trace,base_mean_lfp_trace,stimulus_amplitude_modulation,stimulus_waveform_modulation,lfp_sampling_rate,lfp_times,...,fname,sampling_rate,resistance,capacitance,amplitude_modulation,waveform_modulation,value_max,value_min,waveform,base_waveform
0,2,1,"[[[0.0075687635792413555, 0.001465247922390444...","[0.002112391602832647, -0.000373565219686537, ...","[[[-0.0064875116399061294, -0.0003839960149061...","[-0.002800298964392496, 0.001037548106306116, ...",0.23003291748409427,2.832968816779413e-05,3.3299999999999996e-05,"[-0.0050282999999999994, -0.004994999999999999...",...,"""ec_2_190523.RAF""",2500000,220000,7.500000e-12,0.230033,0.000028,0.433963,-0.795096,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,2,1,"[[[0.008715881544035242, 0.010394360380762135,...","[-0.002484738352135792, -0.0010616228190160827...","[[[0.004629263884159851, 0.009417591612474768,...","[-0.0009374321089675314, 0.0013796302872096378...",0.23003291748409427,2.832968816779413e-05,3.3299999999999996e-05,"[-0.0050282999999999994, -0.004994999999999999...",...,"""ec_2_190523.RAF""",2500000,220000,7.500000e-12,0.230033,0.000028,0.433963,-0.795096,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,3,1,"[[[-0.011763136026413462, -0.00853689793870132...","[-0.003831753522231177, -0.004108510924651803,...","[[[-0.004977901586282474, -0.00728493195901927...","[-0.000850468239671491, -0.0011769891838249308...",0.21003969607302797,-8.781328860030246e-05,3.3299999999999996e-05,"[-0.0050282999999999994, -0.004994999999999999...",...,"""ec_3_190523.RAF""",2500000,280000,1.100000e-11,0.210040,-0.000088,0.426877,-0.782205,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,1,"[[[-0.0030012320202070786, 5.052579229292142e-...","[-0.0016666003784932592, 9.103653512374393e-05...","[[[-0.008366687628595354, -0.00881647540983701...","[-0.0009961245353088677, -0.003053874928829698...",0.21003969607302797,-8.781328860030246e-05,3.3299999999999996e-05,"[-0.0050282999999999994, -0.004994999999999999...",...,"""ec_3_190523.RAF""",2500000,280000,1.100000e-11,0.210040,-0.000088,0.426877,-0.782205,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,1,"[[[0.0003043063693516923, -0.00257789581194178...","[-0.0033268564083073727, -0.003203784209533403...","[[[-0.00035657508942824045, -0.002206989108808...","[0.0005160162949952168, 0.0005936553222554361,...",0.2300875781350591,-0.00923079458871634,3.3299999999999996e-05,"[-0.0050282999999999994, -0.004994999999999999...",...,"""ec_4_190523.RAF""",2500000,280000,4.700000e-11,0.230088,-0.009231,0.431374,-0.797740,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
